In [4]:
import pandas as pd
import numpy as np

In [5]:
from typing import List, Union, Any, Optional, Tuple
import math

def longest_valid_subsequence(values: List[any]) -> Tuple[int, int]:
    """
    This function finds the longest contiguous subsequence without missing/NaN values.
    params:
        values (List[any]): List of values which may contain NaN or None.
    returns:
        Tuple[int, int]: A tuple containing the start and end indices of the longest valid subsequence.
    """

    # Handle Edge Case: Empty List
    if not values:
        return (0, 0)

    def is_valid(value:Any) -> bool:
        """
        This function checks if the value is valud or not i.e NaN or None.
        params:
            value (Any): The value to check.
        returns:
            bool: True if the value is valid (not NaN or None), False otherwise.
        """
        try:
            # Handles both NaN and None
            return not pd.isna(value)
        except TypeError:
            # If the value is not NaN, we take it to be a valid value
            return True
    
    # Initialize variables to track the longest valid subsequence
    max_length = 0
    max_start = 0
    max_end = 0

    current_start = 0
    current_length = 0

    for i, value in enumerate(values):
        if is_valid(value):
            if current_length == 0:  # Starting a new subsequence
                current_start = i
            current_length += 1
        
        else:
            # A invalid value encountered, check if current subsequence is the longest
            if current_length > max_length:
                max_length = current_length
                max_start = current_start
                max_end = current_start + current_length
            
            current_length = 0
            
    # Check final sequence(In case: the longest sequence ends at the last element)
    if current_length > max_length:
        max_length = current_length
        max_start = current_start
        max_end = current_start + current_length
    
    # Return the start and end indices of the longest valid subsequence
    return max_start, max_end
        
    

In [6]:
def find_longest_non_null_subsequence(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function finds the longest subsequence of non-null values for each column in the DataFrame.

    Params:
    df (pd.DataFrame): Input DataFrame with potential null values.
    Returns:
    pd.DataFrame: DataFrame containing the longest non-null subsequence for each column.
    """

    # Create a new dataframe to store the results where rows are columns of the original dataframe
    result_df = pd.DataFrame(columns=['Column', 'Start Index', 'End Index'])

    # Iterate through each column in the DataFrame
    for column in df.columns:
        # Extract the column values
        values = df[column].tolist()
        
        # Find the longest valid subsequence
        start_index, end_index = longest_valid_subsequence(values)
        
        # Add the result to the result DataFrame
        new_row = pd.DataFrame([{
        'Column': column,
        'Start Index': start_index,
        'End Index': end_index
        }])
        result_df = pd.concat([result_df, new_row], ignore_index=True)
        
    # Reset the index of the result DataFrame
    result_df.reset_index(drop=True, inplace=True)

    return result_df

# Call the function on electricity_cleaned dataset
electricity_data = pd.read_csv('electricity_cleaned.csv')
result_df = find_longest_non_null_subsequence(electricity_data)

# Save the result to a CSV file
result_df.to_csv('longest_non_null_subsequence.csv', index=False)

print("Longest non-null subsequence for each column has been saved to 'longest_non_null_subsequence.csv'.")
print(result_df.head())  # Display the first few rows of the result DataFrame

    

Longest non-null subsequence for each column has been saved to 'longest_non_null_subsequence.csv'.
                     Column Start Index End Index
0                 timestamp           0     17544
1  Panther_parking_Lorriane        3378      9860
2      Panther_lodging_Cora        3378      9859
3     Panther_office_Hannah        3733      9859
4    Panther_lodging_Hattie        3378      9859


In [10]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from typing import Tuple, Optional
import warnings

def plot_anomaly_detection_heatmap(df: pd.DataFrame, column: str, timestamp_col: str = 'timestamp') -> None:
    """
    Creates interactive heatmaps showing hourly data for each day across 2 years.
    
    Args:
        df (pd.DataFrame): DataFrame containing the data with a timestamp column.
        column (str): Name of the column to plot
        timestamp_col (str): Name of the timestamp column (default: 'timestamp')
    """
    
    # Make a copy to avoid modifying original dataframe
    data = df.copy()
    
    # Ensure the timestamp column is in datetime format
    data[timestamp_col] = pd.to_datetime(data[timestamp_col], errors='coerce')
    
    # Extract date components
    data['year'] = data[timestamp_col].dt.year
    data['month'] = data[timestamp_col].dt.month
    data['day'] = data[timestamp_col].dt.day
    data['hour'] = data[timestamp_col].dt.hour
    data['date'] = data[timestamp_col].dt.date
    data['day_of_year'] = data[timestamp_col].dt.dayofyear
    
    # Get unique years
    years = sorted(data['year'].unique())
    
    # Use original values without log transformation
    data['values'] = data[column]
    
    # Calculate anomaly thresholds using percentiles
    valid_values = data['values'].dropna()
    if len(valid_values) == 0:
        raise ValueError(f"No valid values found in column '{column}'")
    
    # Define thresholds for anomalies - simplified
    low_threshold = valid_values.quantile(0.25)   # Bottom 25%
    high_threshold = valid_values.quantile(0.75)  # Top 75%
    
    # Create subplot layout
    fig = make_subplots(
        rows=len(years), cols=1,
        subplot_titles=[f'Year {year}' for year in years],
        vertical_spacing=0.05,
        shared_xaxes=True
    )
    
    # Month names and positions for x-axis ticks
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Hour labels
    hour_labels = [f'{i:02d}:00' for i in range(24)]
    
    for year_idx, year in enumerate(years):
        year_data = data[data['year'] == year].copy()
        
        # Sort by timestamp to ensure proper ordering
        year_data = year_data.sort_values(timestamp_col)
        
        # Create pivot table for heatmap - using day_of_year to show all days
        pivot_values = year_data.pivot_table(
            index='hour', 
            columns='day_of_year', 
            values='values', 
            aggfunc='first'  # Take first value if multiple (shouldn't happen with hourly data)
        )
        
        # Create pivot for dates for hover text
        pivot_dates = year_data.pivot_table(
            index='hour', 
            columns='day_of_year', 
            values='date', 
            aggfunc='first'
        )
        
        # Ensure we have all hours (0-23)
        all_hours = list(range(24))
        pivot_values = pivot_values.reindex(index=all_hours)
        pivot_dates = pivot_dates.reindex(index=all_hours)
        
        # Simple two-color colorscale: Blue (low) to Red (high)
        colorscale = [
            [0.0, '#0000FF'],    # Blue - low values
            [1.0, '#FF0000']     # Red - high values
        ]
        
        # Create hover text
        hover_text = []
        for hour in range(24):
            hover_row = []
            for day_col in pivot_values.columns:
                value = pivot_values.loc[hour, day_col] if day_col in pivot_values.columns else np.nan
                date_val = pivot_dates.loc[hour, day_col] if day_col in pivot_dates.columns else None
                
                if pd.isna(value):
                    hover_text_cell = f"Time: {hour:02d}:00<br>Date: {date_val}<br>Value: No Data"
                else:
                    # Determine status based on thresholds
                    if value <= low_threshold:
                        anomaly_status = "Low"
                    elif value >= high_threshold:
                        anomaly_status = "High"
                    else:
                        anomaly_status = "Normal"
                    
                    hover_text_cell = (f"Time: {hour:02d}:00<br>"
                                     f"Date: {date_val}<br>"
                                     f"Value: {value:.2f}<br>"
                                     f"Status: {anomaly_status}")
                
                hover_row.append(hover_text_cell)
            hover_text.append(hover_row)
        
        # Create x-axis labels (day of year)
        x_labels = list(pivot_values.columns)
        
        # Add heatmap to subplot
        fig.add_trace(
            go.Heatmap(
                z=pivot_values.values,
                x=x_labels,
                y=hour_labels,
                colorscale=colorscale,
                hovertemplate='%{customdata}<extra></extra>',
                customdata=hover_text,
                showscale=True if year_idx == len(years)-1 else False,  # Show colorbar only for last subplot
                colorbar=dict(
                    title=f"{column}",
                    titleside="right",
                    x=1.02,
                    len=0.5,
                    y=0.25
                ) if year_idx == len(years)-1 else None,
                zmin=valid_values.min(),
                zmax=valid_values.max()
            ),
            row=year_idx+1, col=1
        )
        
        # Add month boundary lines and labels
        month_boundaries = []
        month_positions = []
        
        for month in range(1, 13):
            month_data = year_data[year_data['month'] == month]
            if not month_data.empty:
                first_day = month_data['day_of_year'].min()
                month_boundaries.append(first_day)
                # Position label in middle of month
                if month < 12:
                    next_month_data = year_data[year_data['month'] == month + 1]
                    if not next_month_data.empty:
                        last_day = next_month_data['day_of_year'].min() - 1
                    else:
                        last_day = month_data['day_of_year'].max()
                else:
                    last_day = month_data['day_of_year'].max()
                
                month_positions.append((first_day + last_day) / 2)
        
        # Update x-axis for this subplot
        fig.update_xaxes(
            title_text="Day of Year" if year_idx == len(years)-1 else "",
            tickmode='array',
            tickvals=month_positions,
            ticktext=month_names[:len(month_positions)],
            row=year_idx+1, col=1
        )
        
        # Update y-axis
        fig.update_yaxes(
            title_text="Hour of Day",
            row=year_idx+1, col=1
        )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'Interactive Daily Heatmap: {column} by Hour and Day<br><sub>Blue (Low Values) → Red (High Values)</sub>',
            x=0.5,
            font=dict(size=14)
        ),
        width=1200,
        height=400 * len(years),
        font=dict(size=10)
    )
    
    # Show the plot
    fig.show()
    
    # Print summary statistics
    print(f"\n=== Summary Statistics for {column} ===")
    print(f"Total data points: {len(data)}")
    print(f"Valid values: {len(valid_values)}")
    print(f"Missing/NaN values: {len(data) - len(valid_values)}")
    print(f"Years covered: {years}")
    print(f"Value range: {valid_values.min():.2f} to {valid_values.max():.2f}")
    print(f"Thresholds:")
    print(f"  Low (25th percentile): < {low_threshold:.2f}")
    print(f"  Normal: {low_threshold:.2f} - {high_threshold:.2f}")
    print(f"  High (75th percentile): > {high_threshold:.2f}")
    
    return fig


column_name = electricity_data.columns[1]

# Call the function to plot the heatmap for the specified column
fig = plot_anomaly_detection_heatmap(electricity_data, column=column_name)
# Save the figure as an HTML file
fig.write_html('anomaly_detection_heatmap.html')
print(f"Anomaly detection heatmap for '{column_name}' has been saved to 'anomaly_detection_heatmap.html'.")


=== Summary Statistics for Panther_parking_Lorriane ===
Total data points: 17544
Valid values: 14162
Missing/NaN values: 3382
Years covered: [2016, 2017]
Value range: 2.10 to 25.50
Thresholds:
  Low (25th percentile): < 8.58
  Normal: 8.58 - 12.90
  High (75th percentile): > 12.90
Anomaly detection heatmap for 'Panther_parking_Lorriane' has been saved to 'anomaly_detection_heatmap.html'.
